In [ ]:
import pyodbc
import requests
import pandas as pd
from io import StringIO


url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=NVDA&outputsize=full&datatype=csv&apikey=TMANXZ3HESIZ8T0U'
r = requests.get(url)

print(r.text)

# 2. Ler direto em um DataFrame (sem salvar em disco)
df = pd.read_csv(StringIO(r.text))

# 3. Tratar dados
df.drop_duplicates(inplace=True)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['year'] = df['timestamp'].dt.year
df['month'] = df['timestamp'].dt.month
df['day'] = df['timestamp'].dt.day
df['time'] = df['timestamp'].dt.time
df = df.drop(columns=['timestamp'])

print(df.head())  # só para inspecionar

# Conexão
server = "ekp-svr-datafinance.database.windows.net"
database = "BD_Dados_Finance"
username = "admin-db"
password = "Pedrica1009@"
driver = '{ODBC Driver 18 for SQL Server}'  # certifique-se que esse driver está instalado

#Conectar
conn = pyodbc.connect(
    f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;'
)

cursor = conn.cursor()

# 5. Inserir no banco
for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO acoes (symbol, data, abertura, maxima, minima, fechamento, volume, year, month, day)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE
            abertura=VALUES(abertura),
            maxima=VALUES(maxima),
            minima=VALUES(minima),
            fechamento=VALUES(fechamento),
            volume=VALUES(volume)
    """, (
        "NVDA", 
        row['timestamp'],   # aqui vc pode manter a coluna timestamp se preferir
        row['open'],
        row['high'],
        row['low'],
        row['close'],
        row['volume'],
        row['year'],
        row['month'],
        row['day']
    ))

conexao.commit()
cursor.close()
conexao.close()

print("✅ Dados tratados e gravados no banco!")


{
    "Information": "We have detected your API key as TMANXZ3HESIZ8T0U and our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits."
}


NameError: name 'ano' is not defined